### Part 2: Data modeling 
#### Assuming we are starting with the data set from Part 1 as our raw data table, how would you model this data in a data warehouse for analytical purposes? What tables would you create? What kinds of questions do you imagine business users would want to ask of this data, and how would they express them in your data model? Please use whatever tools you are comfortable with to answer this question and whatever flavor of SQL you are most familiar with. A github repo or gist is preferred. 

In [1]:
# import libraries

import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 30)

import sqlite3

In [2]:
# import csv
clean_df = pd.read_csv('clean_assignment_log.csv')

In [3]:
# connect to database
conn = sqlite3.connect(':memory:')

# create cursor object
cur = conn.cursor()

In [4]:
# write records stored in the dataframe to SQL database
df = pd.read_csv('clean_assignment_log.csv')
df.to_sql('assignment_log', con = conn, if_exists='append', index=False)

/Users/tararosen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2712: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [5]:
# removing spaces from column names
cur.execute('''SELECT  record,
                       Analyst AS analyst, 
                      "Quality score (sourcing)" AS quality_score_sourcing,
                      "Quality score (writing)" AS quality_score_writing, 
                      "Assigned Job" AS assigned_job,
                      "Accepted Job" AS accepted_job ,
                      "Declined Job" AS declined_job,
                       Action AS action,
                       Request AS request,
                      "Request created at" AS request_created_at,
                       Job AS job,
                      "Event occurred at" AS event_occured_at,
                      "Wait time (min)" AS wait_time_min, 
                      "Waiting for" AS waiting_for, 
                      "Analysts available" AS analysts_available,
                      "Analysts occupied" AS analysts_occupied, 
                      "Total jobs available" AS total_jobs_available, 
                      "Review jobs available" AS review_jobs_available,
                      "Vetting jobs available" AS vetting_jobs_avaiable, 
                      "Planning jobs available" AS planning_jobs_available,
                      "Editing jobs available" AS editing_jobs_available, 
                      "Sourcing jobs available" AS sourcing_jobs_available,
                      "Writing jobs available" AS writing_jobs_available
                       
                FROM assignment_log;''')               # sql query
assign_df = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
assign_df.columns = [x[0] for x in cur.description]    # labels dataframe columns
assign_df.head()                                       # view dataframe

,record,analyst,quality_score_sourcing,quality_score_writing,assigned_job,accepted_job,declined_job,action,request,request_created_at,job,event_occured_at,wait_time_min,waiting_for,analysts_available,analysts_occupied,total_jobs_available,review_jobs_available,vetting_jobs_avaiable,planning_jobs_available,editing_jobs_available,sourcing_jobs_available,writing_jobs_available
0,1,a09c8906073b4c0b75e3100b857b982a,5.0,5.0,0,1,0,Accepted Job,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:50:31,1,"review, planning, editing, sourcing, writing",1,21,13,6,1,6,0,0,0
1,2,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,1,0,0,Assigned Job,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:26:17,1,review,4,21,8,5,1,0,0,1,1
2,3,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,0,1,0,Accepted Job,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:27:19,2,review,1,22,6,5,1,0,0,0,0
3,4,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,1,0,0,Assigned Job,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:26:52,2,review,2,22,7,6,1,0,0,0,0
4,5,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,0,0,1,Declined Job,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:26:52,2,review,2,22,7,6,1,0,0,0,0


In [6]:
# save analyst csv to a file
assign_df.to_csv(r'master_table.csv', index = None, header=True)                  

In [7]:
assign_df.head(1)

,record,analyst,quality_score_sourcing,quality_score_writing,assigned_job,accepted_job,declined_job,action,request,request_created_at,job,event_occured_at,wait_time_min,waiting_for,analysts_available,analysts_occupied,total_jobs_available,review_jobs_available,vetting_jobs_avaiable,planning_jobs_available,editing_jobs_available,sourcing_jobs_available,writing_jobs_available
0,1,a09c8906073b4c0b75e3100b857b982a,5.0,5.0,0,1,0,Accepted Job,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:50:31,1,"review, planning, editing, sourcing, writing",1,21,13,6,1,6,0,0,0


In [8]:
# write records stored in the dataframe to SQL database
df = pd.read_csv('master_table.csv')
df.to_sql('master_table', con = conn, if_exists='append', index=None)

#### Create Analyst Metric Table

In [9]:
# create SQL database for analysts

cur.execute('''SELECT record,
                      analyst, 
                      quality_score_sourcing, 
                      quality_score_writing	,
                      assigned_job,
                      accepted_job,
                      declined_job
               FROM master_table
               ORDER BY record''')                        # sql query
analyst_rank = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
analyst_rank.columns = [x[0] for x in cur.description]    # labels dataframe columns
analyst_rank.head()                                       # view dataframe

,record,analyst,quality_score_sourcing,quality_score_writing,assigned_job,accepted_job,declined_job
0,1,a09c8906073b4c0b75e3100b857b982a,5.0,5.0,0,1,0
1,2,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,1,0,0
2,3,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,0,1,0
3,4,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,1,0,0
4,5,d8e25a290ea51352bf9100a99c475f6d,5.0,5.0,0,0,1


In [10]:
# save analyst csv to a file
analyst_rank.to_csv (r'analyst_table.csv', index = None, header=True) 

In [11]:
# write records stored in the dataframe to SQL database
df = pd.read_csv('analyst_table.csv')
df.to_sql('analyst_table', con = conn, if_exists='append', index=False)

In [12]:
# SQL query to evaluate analyst metrics (sourcing and writing scores,
# and jobs assigned/accepted/declined) 

cur.execute('''SELECT analyst, 
                      AVG(quality_score_sourcing) AS average_sourcing_score, 
                      AVG(quality_score_writing) AS average_writing_score,
                      SUM(assigned_job) AS assigned_job,
                      SUM(accepted_job) AS accepted_job,
                      SUM(declined_job) AS declined_job
               FROM analyst_table
               GROUP BY analyst
               ORDER BY 6 DESC''')                        # sql query
analyst_rank = pd.DataFrame(cur.fetchall())               # create dataframe from sql query
analyst_rank.columns = [x[0] for x in cur.description]    # labels dataframe columns
analyst_rank.head()                                       # view dataframe

,analyst,average_sourcing_score,average_writing_score,assigned_job,accepted_job,declined_job
0,62060850630c7afe54fd59151413d237,4.820000,4.820000,23,11,12
1,85c7b78e76b5232cd38014ea4cdc8f56,4.455882,4.455882,16,8,10
2,a09c8906073b4c0b75e3100b857b982a,5.000000,5.000000,36,28,8
3,c9401f82fdedf3cd080a03bafe3447c7,0.000000,0.000000,9,1,7
4,dddcdea13df8232803957a5d9c847aae,4.310000,4.310000,7,2,4


#### Create Request Table

In [13]:
# create SQL database for requests

cur.execute('''SELECT record,
                      request,
                      request_created_at, 
                      job, 
                      event_occured_at, 
                      wait_time_min,
                      waiting_for
               FROM master_table
               ORDER BY record''')                           # sql query
requests = pd.DataFrame(cur.fetchall())                      # create dataframe from sql query
requests.columns = [x[0] for x in cur.description]           # labels dataframe columns
requests.head()                                              # view dataframe

,record,request,request_created_at,job,event_occured_at,wait_time_min,waiting_for
0,1,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:50:31,1,"review, planning, editing, sourcing, writing"
1,2,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:26:17,1,review
2,3,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:27:19,2,review
3,4,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:26:52,2,review
4,5,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:26:52,2,review


In [14]:
# save request csv to a file
requests.to_csv (r'request_table.csv', index = None, header=True) 

In [15]:
# write records stored in the dataframe to SQL database
df = pd.read_csv('request_table.csv')
df.to_sql('request_table', con = conn, if_exists='append', index=False)

In [16]:
# SQL query to find which requests are still in review, planning, editing, sourcing, writing

cur.execute('''SELECT *
               FROM request_table
               WHERE waiting_for = 'review, planning, editing, sourcing, writing'
               GROUP BY request''')                             # sql query
waiting_for = pd.DataFrame(cur.fetchall())                      # create dataframe from sql query
waiting_for.columns = [x[0] for x in cur.description]           # labels dataframe columns
waiting_for.head()                                                     # view dataframe

,record,request,request_created_at,job,event_occured_at,wait_time_min,waiting_for
0,1,59480d57e759070028da6467,2017-06-19 13:43:51,review,2017-06-21 21:50:31,1,"review, planning, editing, sourcing, writing"
1,13,59485c262e71030033104e3c,2017-06-19 19:20:06,source review,2017-06-22 01:39:39,1,"review, planning, editing, sourcing, writing"
2,25,59494c13d9ae5200633f9695,2017-06-20 12:23:47,review,2017-06-21 22:00:10,1,"review, planning, editing, sourcing, writing"
3,38,5949a81c4d1319005556396c,2017-06-20 18:56:28,source review,2017-06-22 00:58:58,1,"review, planning, editing, sourcing, writing"
4,79,594a5e5566cb69006fdcf09d,2017-06-21 07:53:57,review,2017-06-22 01:18:51,1,"review, planning, editing, sourcing, writing"


#### Create Assigments Status

In [17]:
# create SQL database for analyst availability

cur.execute('''SELECT record,
                      event_occured_at,
                      analysts_available, 
                      analysts_occupied,
                      total_jobs_available, 
                      review_jobs_available,
                      vetting_jobs_avaiable, 
                      planning_jobs_available,
                      editing_jobs_available, 
                      sourcing_jobs_available,
                      writing_jobs_available
               FROM master_table
               ORDER BY event_occured_at DESC''')                        # sql query
availability = pd.DataFrame(cur.fetchall())                              # create dataframe from sql query
availability.columns = [x[0] for x in cur.description]                   # labels dataframe columns
availability.head()                                                      # view dataframe

,record,event_occured_at,analysts_available,analysts_occupied,total_jobs_available,review_jobs_available,vetting_jobs_avaiable,planning_jobs_available,editing_jobs_available,sourcing_jobs_available,writing_jobs_available
0,608,2017-06-22 19:59:06,0,13,14,4,6,2,0,1,1
1,606,2017-06-22 19:59:02,1,13,15,5,6,2,0,1,1
2,663,2017-06-22 19:51:30,1,12,12,5,5,1,0,0,1
3,774,2017-06-22 19:51:01,1,11,12,5,5,1,0,0,1
4,673,2017-06-22 19:50:58,2,11,14,5,5,2,0,1,1


In [18]:
# save availability csv to a file
availability.to_csv (r'availability_table.csv', index = None, header=True) 

In [19]:
# write records stored in the dataframe to SQL database
df = pd.read_csv('availability_table.csv')
df.to_sql('availability_table', con = conn, if_exists='append', index=False)

In [20]:
# create SQL database for analyst availability

cur.execute('''SELECT record,
                      event_occured_at,
                      analysts_available -  total_jobs_available AS position_deficit 
               FROM availability_table
               ORDER BY event_occured_at DESC''')                             # sql query
availability = pd.DataFrame(cur.fetchall())                              # create dataframe from sql query
availability.columns = [x[0] for x in cur.description]                   # labels dataframe columns
availability.head()                                                      # view dataframe

,record,event_occured_at,position_deficit
0,608,2017-06-22 19:59:06,-14
1,606,2017-06-22 19:59:02,-14
2,663,2017-06-22 19:51:30,-11
3,774,2017-06-22 19:51:01,-11
4,673,2017-06-22 19:50:58,-12


In [21]:
conn.close()